In [56]:
import random, string, textwrap

def rand_seq(N):
    return ''.join(random.choices("AGCT", k=N))

#s=rand_seq(1500, 80)

def rand_fasta_file(filename, seq_length, width, size):
    f = open(filename, "w")
    i = 0
    while i < size:
        s = rand_seq(seq_length)
        f.write('>S' + format(i, '05d') + ' randomized    test data\n')
        #f.write('>' +'\n')
        f.write('\n'.join(textwrap.wrap(s, width)))
        f.write('\n')
        i += 1
    f.close()
    
rand_fasta_file("random-1.fa", 1500, 80, 10000)
rand_fasta_file("random-2.fa", 1500, 80, 10000)

In [1]:
from bioinformatics import na_read
from bioinformatics import KmerVectors as kvec
from bioinformatics import NCBIDataset as nds
from bioinformatics import FASTADataset as fads
import pandas as pd

In [2]:
#PATH = "../data/bioinformatics/"
#!ls -al $PATH
TAX_PATH="../data/bioinformatics/ncbi/genome/taxonomy/"
#!ls -al $TAX_PATH
RDP_PATH="../data/bioinformatics/rdp/202208/"
#!ls -al $RDP_PATH

In [3]:
#TAX_VIRUS_PATH="../../../../data/ncbi/genome/taxonomy/Viruses_10239/"
TAX_VIRUS_PATH= TAX_PATH + "Viruses_10239/"

adnaviria_dir    = TAX_VIRUS_PATH + "Adnaviria_2840022/"
bamfordvirae_dir = TAX_VIRUS_PATH + "Bamfordvirae_2732005/Nucleocytoviricota_2732007/Pokkesviricetes_2732525/Chitovirales_2732527/Poxviridae_10240/Orthopoxvirus_10242/"
riboviria_dir    = TAX_VIRUS_PATH + "Riboviria_2559587/Orthornavirae_2732396/Pisuviricota_2732408/Pisoniviricetes_2732506/Nidovirales_76804/Cornidovirineae_2499399/Coronaviridae_11118/Orthocoronavirinae_2501931/"
#"Riboviria_2559587/"

adnaviria_dataset    = adnaviria_dir     + "ncbi_dataset.zip"   #
bamfordvirae_dataset = bamfordvirae_dir  + "ncbi_dataset.zip"   # pox
riboviria_dataset    = riboviria_dir     + "ncbi_dataset.zip"   # sars-cov2

In [4]:
adnaviria_nds = nds.NCBIDataset('adnaviria', adnaviria_dataset)
#adnaviria_nds.dataset_summary()

bamfordvirae_nds = nds.NCBIDataset('bamfordvirae', bamfordvirae_dataset)
#bamfordvirae_nds.dataset_summary()

riboviria_nds = nds.NCBIDataset('riboviria', riboviria_dataset)
#riboviria_nds.dataset_summary()

In [5]:
#adnaviria_nds.ncbi_dataset

In [6]:
kv_ncbi = kvec.KmerVectors(['A','G','C','T'], 6, ncbidatasets=[bamfordvirae_nds, riboviria_nds])
print(kv_ncbi.labels)

KmerVectors Object -
alphabet [['A', 'G', 'C', 'T']]
dict: [['AAAAAA', 'AAAAAG', 'AAAAAC', 'AAAAAT']]...[['TTTTTA', 'TTTTTG', 'TTTTTC', 'TTTTTT']]
Labels: [{'bamfordvirae': 1, 'riboviria': 2}]
[bamfordvirae]
[../data/bioinformatics/ncbi/genome/taxonomy/Viruses_10239/Bamfordvirae_2732005/Nucleocytoviricota_2732007/Pokkesviricetes_2732525/Chitovirales_2732527/Poxviridae_10240/Orthopoxvirus_10242/ncbi_dataset.zip]
dataset name: [bamfordvirae]
   fna/fna2 files Total:  [810]
   fna cds_from_genomic files: [324]
   fna2 genomic files :  [486]
   gff files:  [324]
   faa files:  [324]
class labels: [{'bamfordvirae': 1, 'riboviria': 2}]
[riboviria]
[../data/bioinformatics/ncbi/genome/taxonomy/Viruses_10239/Riboviria_2559587/Orthornavirae_2732396/Pisuviricota_2732408/Pisoniviricetes_2732506/Nidovirales_76804/Cornidovirineae_2499399/Coronaviridae_11118/Orthocoronavirinae_2501931/ncbi_dataset.zip]
dataset name: [riboviria]
   fna/fna2 files Total:  [496]
   fna cds_from_genomic files: [227]
   f

In [7]:
d_ncbi = kv_ncbi.seq2KmerSentences(seq_type='fna2', base_count_max=4, length_min=10000, dataset_limit=300)

NCBI Dataset
ncbi dataset: [bamfordvirae]
102030405060708090100110120130140150160170180190200210220230240250260270280290300310320330340capped at [300]
-
Total:                [348]
Using :               [301]
skip_count_seqtype: [0]
skip_count_minlength: [48]
skip_count_alphabet: [0]
ncbi dataset: [riboviria]
102030405060708090100110120130140150160170180190200210220230240250260-
Total:                [269]
Using :               [251]
skip_count_seqtype: [0]
skip_count_minlength: [17]
skip_count_alphabet: [1]


In [8]:
df_ncbi=pd.DataFrame(data=d_ncbi)
df_ncbi

,v1,v2
0,bamfordvirae,TAGTAA AGTAAA GTAAAA TAAAAT AAAATT AAATTA AATT...
1,bamfordvirae,CACGAC ACGACT CGACTG GACTGT ACTGTA CTGTAG TGTA...
2,bamfordvirae,AAGCAC AGCACT GCACTC CACTCA ACTCAT CTCATT TCAT...
3,bamfordvirae,AAGCAC AGCACT GCACTC CACTCA ACTCAT CTCATT TCAT...
4,bamfordvirae,GACTCC ACTCCA CTCCAT TCCATC CCATCA CATCAG ATCA...
...,...,...
546,riboviria,GACAAA ACAAAG CAAAGC AAAGCT AAGCTG AGCTGA GCTG...
547,riboviria,AGTAAA GTAAAG TAAAGT AAAGTG AAGTGA AGTGAG GTGA...
548,riboviria,CGGGTT GGGTTT GGTTTT GTTTTT TTTTTT TTTTTG TTTT...
549,riboviria,ACTTTT CTTTTA TTTTAA TTTAAA TTAAAG TAAAGT AAAG...


In [9]:
#!ls -al $RDP_PATH
archaea_file = RDP_PATH + "current_Archaea_unaligned.fa"
bacteria_file = RDP_PATH + "current_Bacteria_unaligned.fa"
fungi_file = RDP_PATH + "current_Fungi_unaligned.fa"
#!ls -al $archaea_file

In [10]:
#archaea = na_read.read_fasta(archaea_file)
archaea_fads = fads.FASTADataset('archaea', archaea_file)
print(f'archaea: [{len(archaea_fads.fasta_dataset)}]')
#archaea.fasta_dataset
#bacteria_fads = fads.FASTADataset('bacteria', bacteria_file)
#print(f'bacteria: [{len(bacteria_fads.fasta_dataset)}]')
fungi_fads = fads.FASTADataset('fungi', fungi_file)
print(f'fungi: [{len(fungi_fads.fasta_dataset)}]')

archaea: [160767]
fungi: [125525]


In [11]:
#archaea.fasta_dataset["S000444351"]

In [12]:
#archaea.fasta_dataset["S000444352"]

In [13]:
kv_fasta = kvec.KmerVectors(['A','G','C','T'], 6, fastadatasets=[archaea_fads, fungi_fads])
print(kv_fasta.labels)

KmerVectors Object -
alphabet [['A', 'G', 'C', 'T']]
dict: [['AAAAAA', 'AAAAAG', 'AAAAAC', 'AAAAAT']]...[['TTTTTA', 'TTTTTG', 'TTTTTC', 'TTTTTT']]
Labels: [{'archaea': 1, 'fungi': 2}]
[archaea]
[../data/bioinformatics/rdp/202208/current_Archaea_unaligned.fa]
[fungi]
[../data/bioinformatics/rdp/202208/current_Fungi_unaligned.fa]
{'archaea': 1, 'fungi': 2}


In [14]:
d_fasta = kv_fasta.seq2KmerSentences(base_count_max=4, length_min=1000, dataset_limit=500)

FASTA Dataset
fasta dataset: [archaea]
1000capped at [500]
-
Total:                [1416]
Using :               [501]
skip_count_minlength: [163]
skip_count_alphabet: [753]
fasta dataset: [fungi]
1000capped at [500]
-
Total:                [1685]
Using :               [501]
skip_count_minlength: [235]
skip_count_alphabet: [950]


In [15]:
df_fasta=pd.DataFrame(data=d_fasta)
df_fasta

,v1,v2
0,archaea,ACTCCG CTCCGG TCCGGT CCGGTT CGGTTG GGTTGA GTTG...
1,archaea,GCTAAC CTAACC TAACCT AACCTG ACCTGC CCTGCC CTGC...
2,archaea,CGCCCC GCCCCG CCCCGC CCCGCC CCGCCG CGCCGC GCCG...
3,archaea,CTGGGT TGGGTC GGGTCC GGTCCG GTCCGC TCCGCC CCGC...
4,archaea,CCCGAC CCGACT CGACTG GACTGC ACTGCT CTGCTA TGCT...
...,...,...
995,fungi,TGTACC GTACCC TACCCG ACCCGC CCCGCT CCGCTG CGCT...
996,fungi,TTGATC TGATCT GATCTC ATCTCA TCTCAA CTCAAA TCAA...
997,fungi,CCTCAG CTCAGA TCAGAT CAGATC AGATCA GATCAG ATCA...
998,fungi,TGTACC GTACCC TACCCG ACCCGC CCCGCT CCGCTG CGCT...
